In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
import wandb
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import joblib
import json
import os

In [2]:
df = pd.read_csv('./data/df_for_ML_new.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)

In [3]:
df.fillna(-1, inplace=True)
df['year_built'] = df['year_built'].astype(int)

In [4]:
X = df.drop('target', axis=1)
y = df['target']

In [5]:
X.head()

,baths,sqft,beds,lat,lon,pool,property_type_condo,property_type_land,property_type_multi_family,property_type_other,property_type_ranch,property_type_single_family,property_type_townhouse,property_type_traditional,state,year_built,remodeled_year,avg_school_rating,schools_qty,avg_school_dist
0,3.5,2900.0,4,35.627887,-77.335173,0,0,0,0,0,0,1,0,0,37,2019,0,3.25,8.0,5.54
1,3.0,1947.0,3,47.699482,-117.190979,0,0,0,0,0,0,1,0,0,53,2019,0,0.27,3.0,1.33
2,2.0,3000.0,3,33.943243,-117.376310,1,0,0,0,0,0,1,0,0,6,1961,1967,0.67,3.0,1.96
3,8.0,6457.0,5,32.702323,-97.347071,0,0,0,0,0,0,1,0,0,48,2006,2006,0.92,4.0,0.75
4,-1.0,-1.0,1,27.041567,-82.136146,0,0,1,0,0,0,0,0,0,12,0,0,0.47,3.0,4.08


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
train_pool = Pool(data=X_train, label=y_train)
test_pool = Pool(data=X_test, label=y_test)

In [8]:
model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=12, verbose=False)
model.fit(train_pool)

In [9]:
y_pred = model.predict(test_pool)

In [10]:
r2 = r2_score(y_test, y_pred)

In [11]:
r2

0.6119651986853735

In [12]:
data = pd.read_csv('data/data_with_coords50000.csv')
texts = data['baths']

In [13]:
texts = texts.fillna('')

In [14]:
from torch.utils.data import DataLoader, Dataset

In [21]:
from transformers import BertTokenizer, DataCollatorWithPadding

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

# Токенизация текста
tokens = texts.apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))


In [22]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=10)
model.train()

# Создание датасета и загрузчика
from torch.utils.data import DataLoader, Dataset

class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {'input_ids': self.texts[idx], 'labels': self.labels[idx]}

dataset = TextDataset(tokens, data['baths'])
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=data_collator)

optimizer = AdamW(model.parameters(), lr=2e-5)

# Обучение модели
for epoch in range(3):
    for batch in dataloader:
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/dwarf/miniconda3/envs/diplom_env_conda/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
import matplotlib.pyplot as plt

model.eval()
predictions = []
for batch in dataloader:
    with torch.no_grad():
        outputs = model(**batch)
        predictions.extend(outputs.logits.argmax(dim=1).tolist())

# Визуализация результатов
plt.hist(predictions, bins=len(set(predictions)))
plt.show()
